In [1]:
import pandas as pd
import numpy as np

In [2]:
import os
os.getcwd()

'C:\\Users\\claud\\Downloads'

In [9]:
df = pd.read_csv("stock_news_top_6_stocks")

In [14]:
df = df.drop(columns  ="Unnamed: 0")

In [22]:
df.title[2]

'Apple Freezes New Business for Pegatron on China Labor Abuse'

## Sentiment Model = distilbert-base-uncased-finetuned-sst-2-english

In [42]:
from transformers import pipeline
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [46]:
df.head()

,date,title,content,ticker
0,2020-11-10T20:35:50+00:00,Stocks mixed as tech sell-off continues,"Steve Sosnick, Interactive Brokers Chief Strat...",AAPL
1,2020-11-13T14:00:00+00:00,LinkedIn Co-founder: the number of ‘tech giant...,LinkedIn Co-founder Reid Hoffman joins 'Influe...,AAPL
2,2020-11-09T09:15:52+00:00,Apple Freezes New Business for Pegatron on Chi...,(Bloomberg) -- Apple Inc. suspended new busine...,AAPL
3,2020-11-03T01:17:24+00:00,Apple to Launch MacBooks With Own Chips Next Week,(Bloomberg) -- Apple Inc.’s 15-year relationsh...,AAPL
4,2020-11-11T16:18:37+00:00,Alibaba Singles’ Day Ends Strongly as Beijing ...,(Bloomberg) -- Alibaba Group Holding Ltd. post...,AAPL


In [ ]:
def text_sentiment(X):
    model_name = "distilbert-base-uncased-finetuned-sst-2-english"
    classifier = pipeline("sentiment-analysis", model=model_name)
    res = classifier(X)
    return (res[0]["label"], res[0]["score"])

df["sentiment"] = df["title"].apply(text_sentiment)


In [ ]:
df.head()

In [37]:
import urllib
import csv
task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]


## classification by cardiffnlp/twitter-roberta-base-sentiment

In [40]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
 
 
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
#model = AutoModelForSequenceClassification.from_pretrained(MODEL)
# #model.save_pretrained(MODEL)

# text = "Good night 😊"
# text = preprocess(text)
# encoded_input = tokenizer(text, return_tensors='pt')
# output = model(**encoded_input)
# scores = output[0][0].detach().numpy()
# scores = softmax(scores)

# TF
model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

text = "Good night 😊"
encoded_input = tokenizer(text, return_tensors='tf')
output = model(encoded_input)
scores = output[0][0].numpy()
scores = softmax(scores)

ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


1) positive 0.8466
2) neutral 0.1458
3) negative 0.0076
